In [43]:
from lyricsgenius import Genius
from collections import Counter
import re
from nltk.tokenize import TreebankWordTokenizer
import string
import pandas as pd
import requests
from tqdm.notebook import tqdm
import pickle
import os
import requests
import re
from bs4 import BeautifulSoup
from time import time
from utils import *
import unidecode


#TODO: REPLACE WITH LOCAL PATH
path = r'REPLACE WITH LOCAL PATH'

tokenizer = TreebankWordTokenizer()
punct = set(string.punctuation)
punct.update({"''", "``", "’"})
#TODO: REPLACE WITH YOUR OWN KEY
key = 'REPLACE WITH YOUR OWN KEY'

In [40]:
class GetLyrics():
    def __init__(self, df, genius_key = key, processed_songs = None):
        self.genius_key = key
        self.genius = Genius(self.genius_key, verbose=False, retries = 5)
        self.df = df
        self.song_lyrics = dict()
        if processed_songs:
            self.processed_songs = processed_songs
        else:
            self.processed_songs = set()
    
    def extract_annotations(self, song_id):
        ants = self.genius.song_annotations(song_id)
        if len(ants) == 0:
            return ""
        
        out = []
        for line in ants:
            for a in line[1]:
                out.append(a[0])
        return " ".join(out).lower()


    def get_lyrics(self, get_annotations = True):
        start_time = time()

        
        artists_not_found, songs_not_found, songs_found = 0, 0, 0
        index = 0
        
        gb = self.df.groupby('primary_artist')
        pbar = tqdm(total = len(gb), desc = "0")
        for artist_name, subset in gb:
            artist_obj = self.genius.search_artist(artist_name, max_songs=0)
            if artist_obj is None:
                print(f"{artist_name} not found")
                artists_not_found += 1
            else:
                for _, row in subset.iterrows():
                    index += 1
                    uri = row['track_id']
                    if uri not in self.processed_songs and uri not in self.song_lyrics:
                        song_name = strip_name(row['track_name'])
                        song_obj = artist_obj.song(song_name)
                        if song_obj is None or not match(artist_name, song_obj.artist):
                            songs_not_found += 1
                        else:
                            lyrics = song_obj.to_text().lower()
                            if get_annotations:
                                lyrics += self.extract_annotations(song_obj.id)
                            lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
                            lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])
                            tokens = [t for t in tokenizer.tokenize(lyrics) if t not in punct]
                            cnt = Counter(tokens)
                            self.song_lyrics[uri] = cnt
                            songs_found += 1 
                if index != 0:
                    success_rate = round(songs_found / index, 4)
                 
                pbar.set_description(f"{index}; {success_rate}", refresh=True)            
            pbar.update(1)
        return self.song_lyrics

In [28]:
#TODO
df = pd.read_csv(path + "REPLACE WITH DATASET NAME")
df['primary_artist'] = [unidecode.unidecode(row['artist_name'].split(",")[0]) for _, row in df.iterrows()]
scraper = GetLyrics(df = df, genius_key = key)
lyrics = scraper.get_lyrics(get_annotations = True)

In [ ]:
pickle.dump(lyrics, open("REPLACE WITH OUTPUT FILE NAME", 'wb'))